# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
M. Zhang  ->  M. Zhang  |  ['M. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']


Arxiv has 84 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.07078
extracting tarball to tmp_2509.07078... done.


Issues with the citations
list index out of range
Retrieving document from  https://arxiv.org/e-print/2509.07535


extracting tarball to tmp_2509.07535...

 done.
Retrieving document from  https://arxiv.org/e-print/2509.07640


...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Fig/LOW_GL_example.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=1\linewidth]{Fig/LOW_GL_example.png}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 46 bibliographic references in tmp_2509.07640/3Dmap.bbl.
Retrieving document from  https://arxiv.org/e-print/2509.07744


extracting tarball to tmp_2509.07744...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.07078-b31b1b.svg)](https://arxiv.org/abs/2509.07078) | **Numerical effects on the stripping of dark matter and stars in IllustrisTNG galaxy groups and clusters**  |
|| M. R. Lovell, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-09-10*|
|*Comments*| *23 pages, 16 figures, to be submitted to MNRAS*|
|**Abstract**|            The stellar haloes and intra-cluster light around galaxies are crucial test beds for dark matter (DM) physics and galaxy formation models. We consider the role that the numerical resolution plays in the modelling of these systems by studying the stripping of satellites in the IllustrisTNG cosmological simulations. We focus on host haloes of total halo mass $M_\mathrm{200c}=10^{12-15}M_{\odot}$ and satellites of stellar mass $>10^{7}$$M_{\odot}$, and compare stellar halo / satellite properties across 9 IllustrisTNG runs with baryonic particle mass resolution between $8.5\times10^4M_{\odot}$ and $7\times10^8$$M_{\odot}$, using a Lagrangian-region technique to identify counterpart satellites across different resolution simulations of the same volume. We publish the corresponding catalogues alongside this paper. We demonstrate that the stripping of DM from satellites that orbit in group- and cluster-mass hosts is largely independent of resolution at least until 90 per cent of their initial mass at infall has been stripped. We do not find evidence for spurious disruption of galaxies due to insufficient resolution for the satellite masses we consider. By contrast, the stripping of stellar mass is strongly resolution-dependent: each factor of 8 improvement in particle stellar mass typically adds 2~Gyr to the stripping time. Improved numerical resolution within the IllustrisTNG model generally results in more compact satellites with larger stellar masses, which in turn generate more centrally concentrated stellar haloes and intra-cluster mass profiles. However, the concomitant increase in stellar mass of both satellites and hosts may still be the cause for the overprediction of the stellar halo mass at large host radii relative to observations seen in some previous studies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.07640-b31b1b.svg)](https://arxiv.org/abs/2509.07640) | **An all-sky 3D dust map Based on Gaia and LAMOST**  |
|| T. Wang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-09-10*|
|*Comments*| *25 pages, 20 figures, 3 tables, published in The Astrophysical Journal Supplement Series (ApJS)*|
|**Abstract**|            We present a comprehensive 3D dust reddening map covering the entire Milky Way, constructed by combining reddening estimates based on LAMOST low-resolution spectra (E(B$-$V)$_{\rm LAMOST}$) with those derived from $Gaia$ XP spectra (E(B$-$V)$_{\rm XP}$), along with revised $Gaia$ distances. E(B$-$V)$_{\rm LAMOST}$ values of $\sim$ 4.6 million unique sources were obtained with the standard-pair analysis using LAMOST DR11 stellar parameters and synthesized $B/V$-band photometry from $Gaia$ XP spectra, showing a typical precision of $\sim$ 0.01 mag. The E(B$-$V)$_{\rm XP}$ from the catalog of \citet{zhang2023}, which was derived using forward modeling of $Gaia$ XP spectra, were cross-validated with E(B$-$V)$_{\rm LAMOST}$, leading to the selection of $\sim$ 150 million high-reliability measurements. The combined dataset achieves a median precision of $\sim$ 0.03 mag for E(B$-$V). To model the reddening -- distance relationship along various lines-of-sight, we implemented a parametric approach that accounts for contributions from the local bubble, diffuse interstellar-medium, and multiple potential molecular clouds. The sky was adaptively partitioned based on stellar density, resulting in angular resolutions ranging from 3.4$^{\prime}$ to 58$^{\prime}$, with about half of the sky having a resolution better than 6.9$^{\prime}$. The reddening precision of our 3D map for individual stars reaches $\sim$ 0.01 mag in most regions at $|b| > 20^\circ$, but degrades to 0.01-0.05 mag at $|b| < 20^\circ$. The map reaches a maximum distance of 3-5 kpc in high-extinction regions with $|b| < 5^\circ$, and extends to 10-15 kpc elsewhere. An interactive platform and Python package have been developed for utilization of the 3D dust map. Available online: this https URL.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.07535-b31b1b.svg)](https://arxiv.org/abs/2509.07535) | **Tidal disruption events in active galactic nuclei: on orbital inclination and Schwarzschild apsidal precession**  |
|| <mark>M. Zhang</mark>, et al. |
|*Appeared on*| *2025-09-10*|
|*Comments*| *27 pages, 16 figures, 2 tables, accepted for publication in ApJ*|
|**Abstract**|            Tidal disruption events (TDEs) in active galactic nuclei (AGNs) mark a regime where traditional vacuum models fail to capture the full dynamics, especially due to interaction between stellar debris and pre-existing accretion disks. We perform meshless hydrodynamic simulations incorporating both general relativistic (GR) effects and radiative cooling to study TDEs in AGNs with different orbital inclinations ($\theta_{\rm inc}$) of the disrupted star, ranging from projected prograde to retrograde orbits. We post-process the simulations to derive multi-wavelength light curves and identify several distinct features in the light curves, including a precursor flare from early debris-disk collision and a major flare driven by fallback. The dynamics of the stellar debris and accretion disk, and subsequently the light curve features, are strongly affected by $\theta_{\rm inc}$ and GR effects. Retrograde orbits ($\theta_{\rm inc}=135^\circ$) yield a more luminous, shorter major flare and a more prominent precursor than prograde ones ($\theta_{\rm inc}=22.5^\circ$). During fallback, prograde cases ($\theta_{\rm inc} = 22.5^\circ$, $45^\circ$) develop a central cavity with spirals in the inner region of the AGN disk, leading to transient UV/X-ray suppression accompanied by oscillations, while higher inclinations ($\theta_{\rm inc}=90^\circ$, $135^\circ$) form a gradually tilting inner disk, potentially causing UV/X-ray dips via geometric effects at certain viewing angles. Relativistic apsidal precession alters stream collisions, producing structural differences in the inner disk, outer disk, and debris compared to Newtonian cases, and drives quasi-periodic signals in prograde configurations. These results provide predictive diagnostics for identifying AGN TDEs and interpreting observed light-curve diversity.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.07744-b31b1b.svg)](https://arxiv.org/abs/2509.07744) | **Unveiling Galactic substructures with M Giant stars: A kinematic and chemical study based on LAMOST DR9, Gaia DR3 and APOGEE DR17**  |
|| L. Ding, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-09-10*|
|*Comments*| **|
|**Abstract**|            Based on the updated M giant star catalog selected from LAMOST DR9, we iden-tify substructures within the integrals-of-motion space through Friends-of-Friends cluster-ing algorithm. We obtain members belonging to several known substructures: the Sagittarius stream, Galactic Anticenter Substructure (GASS), Gaia-Enceladus-Sausage (GES), Splash, and the high-{\alpha} disk. Furthermore, we also identify two groups which cannot be clearly asso-ciated with previously known substructures. Our findings confirm the existence of metal-rich constituents within the GES, representing newly formed stars that originated from the metal-enriched gas delivered during the GES merger event and subsequently evolved. Additionally, this study further expands the sample of GASS, high-{\alpha} disk, and Splash stars. Analysis of these metal-rich M giant stars as members of the GES, Splash, and high-{\alpha} disk compo-nents supports an evolution scenario for the early Milky Way, as proposed by previous stud-ies. In this scenario, stars initially formed in a high-{\alpha} primordial disk were dynamically heated by the massive accretion event (GES). This process redistributed stellar orbits, creat-ing the Splash population, while the undisturbed portion of the primordial disk persisted as the present-day high-{\alpha} disk component.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2509.07078/./figures/InfallEvolExamplesSeed48_DM30_ResHMSM-eps-converted-to.png', 'tmp_2509.07078/./figures/StrippingHalfTime_DM30_ResHMSM_2-eps-converted-to.png', 'tmp_2509.07078/./figures/subhaloMassFct_resTest_allSubs.png']
copying  tmp_2509.07078/./figures/InfallEvolExamplesSeed48_DM30_ResHMSM-eps-converted-to.png to _build/html/
copying  tmp_2509.07078/./figures/StrippingHalfTime_DM30_ResHMSM_2-eps-converted-to.png to _build/html/
copying  tmp_2509.07078/./figures/subhaloMassFct_resTest_allSubs.png to _build/html/
exported in  _build/html/2509.07078.md
    + _build/html/tmp_2509.07078/./figures/InfallEvolExamplesSeed48_DM30_ResHMSM-eps-converted-to.png
    + _build/html/tmp_2509.07078/./figures/StrippingHalfTime_DM30_ResHMSM_2-eps-converted-to.png
    + _build/html/tmp_2509.07078/./figures/subhaloMassFct_resTest_allSubs.png
found figures ['tmp_2509.07640/./Fig/hp_HEALPix.png', 'tmp_2509.07640/./Fig/star_pair_fit_sigma.png', 'tmp_2509.07640/./Fig/SFD_vs_Our.png'

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\sm}{\small}$
$\newcommand{\dint}{ {\rm d}}$
$\newcommand{\hmsun}{ h^{-1} {\rm M_\odot}}$
$\newcommand{\msun}{{ \rm M_\odot}}$
$\newcommand{\vmax}{{ V_{\rm max}~}}$
$\newcommand{\Lsun}{{ \rm L_\odot}}$
$\newcommand{\kms}{ {\rm km} {\rm s}^{-1}}$
$\newcommand{\cm}{ {\rm cm}}$
$\newcommand{\erg}{ {\rm erg}}$
$\newcommand{\Gyr}{ {\rm Gyr}}$
$\newcommand{\K}{ {\rm K}}$
$\newcommand{\Myr}{ {\rm Myr}}$
$\newcommand{\pc}{ {\rm pc}}$
$\newcommand{\kpc}{ {\rm kpc}}$
$\newcommand{\Mpc}{ {\rm Mpc}}$
$\newcommand{\hkpc}{ h^{-1} {\rm kpc}}$
$\newcommand{\hmpc}{ h^{-1} {\rm Mpc}}$
$\newcommand{\cpm}{ {\rm cm}^2 {\rm g}^{-1}}$
$\newcommand{\gcm}{  {\rm g} {\rm cm}^{-3}}$
$\newcommand{\mum}{ \mu{\rm m}}$
$\newcommand{\mstar}{M_{\star}}$
$\newcommand{\ml}[1]{\textcolor{red}{#1}}$
$\newcommand{\ap}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\gsim}{ \lower .75ex \hbox{\sim} \llap{\raise .27ex \hbox{>}} }$
$\newcommand{\lsim}{ \lower .75ex \hbox{\sim} \llap{\raise .27ex \hbox{<}} }$
$\newcommand\MTWOC{M_{\rm 200c}}$
$\newcommand\RTWOC{R_{\rm 200c}}$
$\newcommand\mn{@urlcharsother}$
$\newcommand\mn{@doi}$
$\newcommand\mn{@doi@}$
$\newcommand\mn{@eprint#1#2}$
$\newcommand\mn{@eprint@arXiv#1}$
$\newcommand\mn{@eprint@dblp#1}$
$\newcommand\mn{@eprint@#1:#2:#3:#4}$
$\newcommand{\@}{tempa}$
$\newcommand{\@}{tempa }$
$\newcommand{\@}{tempb }$
$\newcommand{\@}{tempc$
$  }$
$\newcommand{\@}{tempb }$</div>



<div id="title">

# Numerical effects on the stripping of dark matter and stars in IllustrisTNG galaxy groups and clusters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.07078-b31b1b.svg)](https://arxiv.org/abs/2509.07078)<mark>Appeared on: 2025-09-10</mark> -  _23 pages, 16 figures, to be submitted to MNRAS_

</div>
<div id="authors">

M. R. Lovell, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ The stellar haloes and intra-cluster light around galaxies are crucial test beds for dark matter (DM) physics and galaxy formation models. We consider the role that the numerical resolution plays in the modelling of these systems by studying the stripping of satellites in the IllustrisTNG cosmological simulations. We focus on host haloes of total halo mass $\MTWOC=10^{12-15}\msun$ and satellites of stellar mass $>10^{7}$  $\msun$ , and compare stellar halo / satellite properties across 9 IllustrisTNG runs with baryonic particle mass resolution between $8.5\times10^4\msun$ and $7\times10^8$  $\msun$ , using a Lagrangian-region technique to identify counterpart satellites across different resolution simulations of the same volume. We publish the corresponding catalogues alongside this paper. We demonstrate that the stripping of DM from satellites that orbit in group- and cluster-mass hosts is largely independent of resolution at least until 90 per cent of their initial mass at infall has been stripped. We do not find evidence for spurious disruption of galaxies due to insufficient resolution for the satellite masses we consider. By contrast, the stripping of stellar mass is strongly resolution-dependent: each factor of 8 improvement in particle stellar mass typically adds 2 Gyr to the stripping time. Improved numerical resolution within the IllustrisTNG model generally results in more compact satellites with larger stellar masses, which in turn generate more centrally concentrated stellar haloes and intra-cluster mass profiles. However, the concomitant increase in stellar mass of both satellites and hosts may still be the cause for the overprediction of the stellar halo mass at large host radii relative to observations seen in some previous studies.

</div>

<div id="div_fig1">

<img src="tmp_2509.07078/./figures/InfallEvolExamplesSeed48_DM30_ResHMSM-eps-converted-to.png" alt="Fig8" width="100%"/>

**Figure 8. -** The evolution with time of the DM mass in randomly-selected TNG example satellites, starting at infall. Columns (rows) denote bins in satellite stellar mass at infall (host halo mass at $z=0$). Here we show the evolution of satellite galaxies matched across runs at different resolution levels and selected in the highest one (L1, i.e. selected in TNG50-1, TNG100-1 and TNG300-1 as described in Section \ref{sec:stripping_methods}). Results from the L1, L2, and L3 runs are shown as solid, dashed and dot-dashed curves, respectively, with different colours denoting different simulated volumes. The smaller the resolution effects on the stripping of DM from satellites, the smaller is the distance among curves of the same colour. (*fig:IEDM*)

</div>
<div id="div_fig2">

<img src="tmp_2509.07078/./figures/StrippingHalfTime_DM30_ResHMSM_2-eps-converted-to.png" alt="Fig10" width="100%"/>

**Figure 10. -** The time after infall at which galaxies are first stripped to 50 per cent of their infall DM mass as measured within 30 kpc, $t_\rmn{50}$. On the $y$-axis we show the difference in stripping time between high- and low-resolution copies of satellites: L3 minus L2 and L2 minus L1. On the $x$-axis we show the $t_\rmn{50}$ of the higher resolution counterpart. If the number of L3--to--L2 matches in a panel is at least half the number of L2--to--L1 matches, we plot data for both sets of pairs; otherwise we only plot the L2--to--L1 data. Points below the 1:1 line (i.e. the dashed line at ratio equal 0) denote satellite pairs for which the low resolution counterpart is stripped earlier than the higher resolution version, and the opposite is true for points above the 1:1 line. The median data points are shown as large plus symbols (L3 versus L2) and large crosses (L2 versus L1) when binned by the L1 $M_\rmn{*-Infall}$; we also include L3 versus L2 medians when binned by L2 $M_\rmn{*-Infall}$ as empty circles. The diagonal black lines delineate parts of the plotting region that cannot include data, because either $t_\rmn{50}$ of the low resolution counterpart would occur prior to infall or later than 10.5 Gyr, i.e. at a time later than $z=0$. Each panel corresponds to a different halo mass / satellite infall stellar mass bin combination, which are binned by the $M_\rmn{host}$ and $M_\rmn{*-Infall}$ of the L1 counterpart. The relationship between symbol colour and data set combination is given in the bottom right panel figure legend. On average, and barring the worst resolution level at hand (TNG300-3), the same satellite across resolution levels is stripped of its DM mass at similar rates. However, poorer resolution tends to strip faster those satellites that take longer to be stripped (see markers below the zero line and with larger times after infall to be 50 per cent stripped). (*fig:STDM1*)

</div>
<div id="div_fig3">

<img src="tmp_2509.07078/./figures/subhaloMassFct_resTest_allSubs.png" alt="Fig5" width="100%"/>

**Figure 5. -** Subhalo mass functions for six logarithmic bins in host halo mass across the TNG simulation suite at $z=0$. The definition of mass is the total mass bound to each subhalo, here referred to as the dynamical mass, $M_\rmn{dyn}$. Level-1 resolution simulations are shown with solid lines, level-2 with faded dashed lines, and level-3 with faded dot-dashed lines. TNG100, TNG300, and TNG50 results are shown in blue, orange, and magenta, respectively. The host halo masses increase across panels, from left to right and top to bottom; the host mass values are given in each panel legend. (*fig:submf*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.07078"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# An all-sky 3D dust map Based on Gaia and LAMOST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.07640-b31b1b.svg)](https://arxiv.org/abs/2509.07640)<mark>Appeared on: 2025-09-10</mark> -  _25 pages, 20 figures, 3 tables, published in The Astrophysical Journal Supplement Series (ApJS)_

</div>
<div id="authors">

T. Wang (王涛), et al.

</div>
<div id="abstract">

**Abstract:** We present a comprehensive 3D dust reddening map covering the entire Milky Way, constructed by combining reddening estimates based on LAMOST low-resolution spectra (E(B $-$ V) $_{\rm LAMOST}$ ) with those derived from $\gaia$ XP spectra (E(B $-$ V) $_{\rm XP}$ ), along with revised $\gaia$ distances. E(B $-$ V) $_{\rm LAMOST}$ values of $\sim$ 4.6 million unique sources were obtained with the standard-pair analysis using LAMOST DR11 stellar parameters and synthesized $B/V$ -band photometry from $\gaia$ XP spectra, showing a typical precision of $\sim$ 0.01 mag. The E(B $-$ V) $_{\rm XP}$ from the catalog of [Zhang, Green and Rix (2023)]() , which was derived using forward modeling of $\gaia$ XP spectra, were cross-validated with E(B $-$ V) $_{\rm LAMOST}$ , leading to the selection of $\sim$ 150 million high-reliability measurements. The combined dataset achieves a median precision of $\sim$ 0.03 mag for E(B $-$ V). To model the reddening--distance relationship along various lines-of-sight, we implemented a parametric approach that accounts for contributions from the local bubble, diffuse interstellar-medium, and multiple potential molecular clouds. The sky was adaptively partitioned based on stellar density, resulting in angular resolutions ranging from 3.4 $^{\prime}$ to 58 $^{\prime}$ , with about half of the sky having a resolution better than 6.9 $^{\prime}$ . The reddening precision of our 3D map for individual stars reaches $\sim$ 0.01 mag in most regions at $|b| > 20^\circ$ , but degrades to 0.01 -- 0.05 mag at $|b| < 20^\circ$ . The map reaches a maximum distance of 3 -- 5 kpc in high-extinction regions with $|b| < 5^\circ$ , and extends to 10 -- 15 kpc elsewhere. An interactive platform and Python package have been developed for utilization of the 3D dust map.

</div>

<div id="div_fig1">

<img src="tmp_2509.07640/./Fig/hp_HEALPix.png" alt="Fig14" width="100%"/>

**Figure 14. -** Adaptive multi-resolution line-of-sight scheme maximum reliable distance for each line-of-sight. (a) The number density of the LAMOST and XP sample（see subsection \ref{sec:data_set}）in the Galactic coordinate system. The number of stars per $\sim$ 0.21 deg$^2$ area（$\rm N_{side}$ =128）is shown. (b) A adaptive multi-resolution map created using the data shown in panel (a), in the same coordinate system. The map consists of five distinct HEALPix pixels, each represented by a different color: brown represents a resolution of 54$\arcmin$(area $\sim$ 0.84$ \mathrm{deg}^2$), yellow represents 28$\arcmin$(area $\sim$ 0.21$ \mathrm{deg}^2$), blue represents 14$\arcmin$(area $\sim$ 0.05$ \mathrm{deg}^2$), red represents 6.6$\arcmin$(area $\sim$ 0.01$ \mathrm{deg}^2$), and green represents 3.4$\arcmin$(area $\sim$ 0.003$ \mathrm{deg}^2$). (c) The distribution of the number of stars per line-of-sight, derived from the adaptive multi-resolution scheme shown in panel (b), is color-coded according to the stellar count. (d) In the adaptive multi-resolution map, the maximum reliable distance for each line-of-sight is color-coded. (*fig:HEALPix*)

</div>
<div id="div_fig2">

<img src="tmp_2509.07640/./Fig/star_pair_fit_sigma.png" alt="Fig2" width="100%"/>

**Figure 2. -** The 2D histogram of the residual standard deviation from the linear fit to the target stars, derived using the standard-pair algorithm, as a function of $\tefflm$, with color encoding representing the stellar number density. (*fig:star_pair_fit_sigma*)

</div>
<div id="div_fig3">

<img src="tmp_2509.07640/./Fig/SFD_vs_Our.png" alt="Fig9" width="100%"/>

**Figure 9. -** A comparison of the E(B$-$V) derived from the extinction integral at the maximum reliable distance in this work and those from the SFD is shown. The black points represent the median values within each bin, and the black line is the linear fit to these points. \st{Note that regions with |b| > 20$^\circ$ and MCs have been excluded.} LMC, SMC, and |b| < 20$^\circ$ regions were excluded. (*fig:sfd_our*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.07640"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

127  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
